In [1]:
import os
from pathlib import Path
from typing import Optional
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
%matplotlib inline

In [2]:
np.random.seed(42)

In [3]:
df_calendar = pd.read_csv("../data/calendar.csv", parse_dates=["date"])
df_train_val = pd.read_csv("../data/sales_train_validation.csv")
df_prices = pd.read_csv("../data/sell_prices.csv")

In [4]:
df_train_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [5]:
df_train_val.shape

(30490, 1919)

In [6]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int64         
 2   weekday       1969 non-null   object        
 3   wday          1969 non-null   int64         
 4   month         1969 non-null   int64         
 5   year          1969 non-null   int64         
 6   d             1969 non-null   object        
 7   event_name_1  162 non-null    object        
 8   event_type_1  162 non-null    object        
 9   event_name_2  5 non-null      object        
 10  event_type_2  5 non-null      object        
 11  snap_CA       1969 non-null   int64         
 12  snap_TX       1969 non-null   int64         
 13  snap_WI       1969 non-null   int64         
dtypes: datetime64[ns](1), int64(7), object(6)
memory usage: 215.5+ KB


In [7]:
df_calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [8]:
df_calendar.insert(3, "week_of_year", df_calendar['date'].dt.isocalendar().week) 

In [9]:
df_calendar['event_name'] = df_calendar['event_name_1'].apply(lambda s: s if isinstance(s, str) else "")
df_calendar['event'] = df_calendar['event_name'].apply(lambda el: 1 if el else 0)

In [10]:
df_calendar['event_name'].unique()

array(['', 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'StPatricksDay', 'Purim End', 'OrthodoxEaster',
       'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', "Father's day",
       'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay',
       'ColumbusDay', 'Halloween', 'EidAlAdha', 'VeteransDay',
       'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter'], dtype=object)

In [11]:
df_calendar = df_calendar[['date', 'wm_yr_wk', 'weekday', 'week_of_year', 'year', 'd', 'event_name', 'event', "snap_CA",	"snap_TX",	"snap_WI"]]

In [12]:
df_calendar.head()

,date,wm_yr_wk,weekday,week_of_year,year,d,event_name,event,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,4,2011,d_1,,0,0,0,0
1,2011-01-30,11101,Sunday,4,2011,d_2,,0,0,0,0
2,2011-01-31,11101,Monday,5,2011,d_3,,0,0,0,0
3,2011-02-01,11101,Tuesday,5,2011,d_4,,0,1,1,0
4,2011-02-02,11101,Wednesday,5,2011,d_5,,0,1,0,1


In [13]:
df_calendar[df_calendar.columns[df_calendar.columns.str.startswith("snap")]].apply(lambda s: s.value_counts(normalize=True))

,snap_CA,snap_TX,snap_WI
0,0.669883,0.669883,0.669883
1,0.330117,0.330117,0.330117


In [14]:
# SNAP (Supplemental Nutrition Assistance Program) is a U.S. government program that
# helps low-income individuals and families buy food. Walmart accepts SNAP benefits,
# allowing customers to use their Electronic Benefit Transfer (EBT) cards for purchases both in-store and online

In [15]:
df_prices.shape

(6841121, 4)

In [16]:
df_prices['store_id'].value_counts().sort_index()

store_id
CA_1    698412
CA_2    637395
CA_3    693990
CA_4    679025
TX_1    699796
TX_2    701214
TX_3    691112
WI_1    665912
WI_2    678171
WI_3    696094
Name: count, dtype: int64

In [17]:
eda_path = Path(os.getcwd()).parent.joinpath("outputs").joinpath("eda")


In [18]:
os.makedirs(eda_path, exist_ok=True)

In [19]:
# takes only specific department (FOOD_3)
df_prices = df_prices.loc[df_prices['item_id'].str.startswith("FOODS_3")]

In [20]:
df_prices[['sell_price']].describe().to_csv(f'{eda_path}/sell_prices.csv')

In [21]:
df_prices['sell_price'].plot(kind="hist", bins=100, title="Price distribution (Foods_3)")
plt.savefig(f'{eda_path}/sell_prices.png')
plt.close()
# sns.histplot(x="sell_price", data=df_prices, kde=True, bins=100)

In [22]:
df_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
511649,CA_1,FOODS_3_001,11101,2.28
511650,CA_1,FOODS_3_001,11102,2.28
511651,CA_1,FOODS_3_001,11103,2.28
511652,CA_1,FOODS_3_001,11104,2.28
511653,CA_1,FOODS_3_001,11105,2.28


In [23]:
df_train_val = df_train_val.query("dept_id == 'FOODS_3'")

In [24]:
df_train_val.shape

(8230, 1919)

In [25]:
df_train_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
2226,FOODS_3_001_CA_1_validation,FOODS_3_001,FOODS_3,FOODS,CA_1,CA,1,1,1,1,...,0,0,1,2,0,0,1,0,0,1
2227,FOODS_3_002_CA_1_validation,FOODS_3_002,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2228,FOODS_3_003_CA_1_validation,FOODS_3_003,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,0,1,1,1,0,0,0,0,1,0
2229,FOODS_3_004_CA_1_validation,FOODS_3_004,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,1,2,0,0,0,0,0,2,0,1
2230,FOODS_3_005_CA_1_validation,FOODS_3_005,FOODS_3,FOODS,CA_1,CA,1,0,1,2,...,0,0,2,0,0,0,0,0,1,0


In [26]:
df_train_val = df_train_val.set_index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])

In [27]:
df_train_val.isnull().sum().sum().item()

0

In [28]:
fig, ax = plt.subplots(figsize=(6, 4), layout="tight")
(df_train_val == 0).mean(axis=1).plot(kind="hist", bins=100, ax=ax)
ax.set_title("Fraction of Zeros in a Time Series (FOODS_3)")
ax.axvline(x=(df_train_val == 0).mean(axis=1).median().round(3).item(), label="median", linestyle="--", color="orange")
ax.legend()
plt.savefig(eda_path.joinpath("foods_zeros.png"))
plt.close()

In [38]:
df_train_val.sum(axis=1).plot(kind="hist", bins=50)
plt.title("Total demands (FOODS_3)")
plt.savefig(eda_path.joinpath("total_demands.png"))
plt.close()

In [30]:
# demand looks normally distributed across the different items in FOODS_3 in log scale
df_train_val.sum(axis=1).plot(
    kind="hist", logy=True, logx=True, bins=np.logspace(0, 6, 100),
    title="Total Sales Distribution (log scale)", legend=None
)
plt.tight_layout()
plt.savefig(eda_path.joinpath("total_demands_log.png"))
plt.close()

In [31]:
total_daily_sales = df_train_val.sum(axis=0)

In [32]:
total_daily_sales.tail()

d_1909    16241
d_1910    16232
d_1911    18561
d_1912    22691
d_1913    22996
dtype: int64

In [33]:
day2date = df_calendar.set_index('d')['date'].to_dict()

In [34]:
total_daily_sales.index = total_daily_sales.index.map(day2date)
total_daily_sales = total_daily_sales.to_frame(name="sales")

In [39]:
total_daily_sales["sales"].plot()
plt.title("Daily sales (FOODS_3)")
plt.savefig(eda_path.joinpath("daily_sales.png"))
plt.close()

In [32]:
total_daily_sales.loc[total_daily_sales["sales"] < 5_000, ["sales"]]

,sales
2011-12-25,13
2012-12-25,10
2013-12-25,20
2014-12-25,17
2015-12-25,12


In [40]:
pd.merge(
    df_calendar[["date", "event_name"]],
    total_daily_sales.loc[total_daily_sales["sales"] < 5_000, ["sales"]],
    left_on="date",
    right_index=True).to_csv(eda_path.joinpath("daily_low_sales.csv"), index=False)

In [41]:
# zooming in
fig, axes = plt.subplots(figsize=(10, 8), nrows=2, ncols=1, layout="constrained")
(
    total_daily_sales
        .loc[slice(pd.Timestamp("2012-01-01"), pd.Timestamp("2012-12-31"))]
        .plot(ax=axes[0])
)
axes[0].set(xlabel="date", ylabel="daily sales", title="Total Daily Sales in 2012")
axes[0].tick_params(axis="x", rotation=30)
(
    total_daily_sales
        .loc[slice(pd.Timestamp("2012-01-01"), pd.Timestamp("2012-02-01"))]
        .plot(ax=axes[1])
)
axes[1].set(xlabel="date", ylabel="daily sales", title="Total Daily Sales in January 2012")
axes[1].tick_params(axis="x", rotation=30)
plt.savefig(eda_path.joinpath("sales_by_year_and_month.png"))
plt.close()

In [42]:
sales_per_year = total_daily_sales.loc[slice(pd.Timestamp("2012-01-01"), pd.Timestamp("2015-12-31"))].copy()
sales_per_year["month"] = sales_per_year.index.month_name().str.slice(stop=3)
sales_per_year['year'] = sales_per_year.index.year

In [36]:
# total_daily_sales['year'] = total_daily_sales.index.year
# total_daily_sales.head()

In [43]:
fig, ax = plt.subplots(layout="tight")
sns.lineplot(x="month", y="sales",  hue="year", errorbar=None, data=sales_per_year, ax=ax)
fig.autofmt_xdate()
ax.set_title("Total Yearly Sales")
fig.savefig(eda_path.joinpath("total_yearly_sales.png"))
plt.close()

2

In [44]:
state_sales = df_train_val.reset_index(level="state_id").groupby("state_id").sum()
state_sales.columns = state_sales.columns.map(day2date)
# state_sales.columns

In [46]:
fig, ax = plt.subplots(layout="tight")
state_sales.T.plot(alpha=0.7, ax=ax)
ax.set_title("Total Sales by State (FOODS_3)")
fig.autofmt_xdate()
fig.savefig(eda_path.joinpath("total_sales_by_state.png"))
plt.close()

In [42]:
# state_sales.plot()

In [43]:
df_train_val.mean(axis=1).quantile(q=[0.05, 0.1, 0.5, 0.9, 0.95])

0.05    0.107919
0.10    0.170936
0.50    0.812075
0.90    4.442917
0.95    7.663408
dtype: float64

In [47]:
mean_demand_stats = df_train_val.mean(axis=1).agg(
        q_05=lambda s: s.quantile(q=0.05),
        q_10=lambda s: s.quantile(q=0.1),
        q_50=lambda s: s.quantile(q=0.5),        
        q_90=lambda s: s.quantile(q=0.9),
        q_95=lambda s: s.quantile(q=0.95),
        min=lambda s: s.min(),
        mean=lambda s: s.mean(),
        max=lambda s: s.max(),
        SD=lambda s: s.std(),
)
mean_demand_stats.to_csv(eda_path.joinpath("mean_demand_stats.csv"))

In [49]:
sparse_series_idx = (
    df_train_val
    .mean(axis=1)[lambda mean_demand: mean_demand <= mean_demand_stats.loc["q_05"]]
    .sample(5, replace=False)
    .index
)
sparse_series_idx

MultiIndex([('FOODS_3_527_TX_3_validation', 'FOODS_3_527', 'FOODS_3', ...),
            ('FOODS_3_466_CA_3_validation', 'FOODS_3_466', 'FOODS_3', ...),
            ('FOODS_3_165_TX_3_validation', 'FOODS_3_165', 'FOODS_3', ...),
            ('FOODS_3_471_CA_2_validation', 'FOODS_3_471', 'FOODS_3', ...),
            ('FOODS_3_696_TX_3_validation', 'FOODS_3_696', 'FOODS_3', ...)],
           names=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])

In [50]:
df_sparse = df_train_val.loc[sparse_series_idx].copy()
df_sparse = df_sparse.reset_index(level="item_id").set_index("item_id").T
df_sparse.index = df_sparse.index.map(day2date)
fig, axes = plt.subplots(nrows=df_sparse.shape[1], figsize=(12, 4))
df_sparse.plot(ax=axes, subplots=True, legend=False)
for ax in axes.ravel():
    ax.legend(loc="upper left")
fig.autofmt_xdate()
fig.suptitle("Sparse Time Series")
fig.savefig(eda_path.joinpath("sparse_time_series.png"))
plt.close()

In [55]:
mask = np.triu(np.ones_like(df_sparse.corr(), dtype=bool))

# generate heatmap
fig, ax = plt.subplots(figsize=(6,3), layout="tight")
sns.heatmap(df_sparse.corr().round(3), annot=True, mask=mask, vmin=-1, vmax=1, ax=ax)
fig.autofmt_xdate(rotation=45)
ax.set_title('Correlation Matrix')
# plt.tight_layout()
fig.savefig(eda_path.joinpath("sparse_time_series_corr.png"))
plt.close()

In [48]:
df_sparse.corr()

item_id,FOODS_3_527,FOODS_3_466,FOODS_3_165,FOODS_3_471,FOODS_3_696
item_id,,,,,
FOODS_3_527,1.000000,0.036711,0.021319,0.136465,0.065693
FOODS_3_466,0.036711,1.000000,-0.021599,0.065055,0.062192
FOODS_3_165,0.021319,-0.021599,1.000000,0.025326,0.008862
FOODS_3_471,0.136465,0.065055,0.025326,1.000000,0.043654
FOODS_3_696,0.065693,0.062192,0.008862,0.043654,1.000000


In [56]:
delta = 0.01
median_series_idx = (
    df_train_val
    .mean(axis=1)[lambda mean_demand: mean_demand.between(mean_demand_stats.loc["q_50"] - delta, mean_demand_stats.loc["q_50"] + delta)]
    .sample(5, replace=False)
    .index
)
median_series_idx

MultiIndex([('FOODS_3_595_WI_1_validation', 'FOODS_3_595', 'FOODS_3', ...),
            ('FOODS_3_669_CA_1_validation', 'FOODS_3_669', 'FOODS_3', ...),
            ('FOODS_3_200_WI_2_validation', 'FOODS_3_200', 'FOODS_3', ...),
            ('FOODS_3_566_TX_3_validation', 'FOODS_3_566', 'FOODS_3', ...),
            ('FOODS_3_665_CA_4_validation', 'FOODS_3_665', 'FOODS_3', ...)],
           names=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])

In [50]:
df_train_val.head()

,,,,,,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,item_id,dept_id,cat_id,store_id,state_id,,,,,,,,,,,,,,,,,,,,,
FOODS_3_001_CA_1_validation,FOODS_3_001,FOODS_3,FOODS,CA_1,CA,1,1,1,1,1,0,1,2,1,1,...,0,0,1,2,0,0,1,0,0,1
FOODS_3_002_CA_1_validation,FOODS_3_002,FOODS_3,FOODS,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FOODS_3_003_CA_1_validation,FOODS_3_003,FOODS_3,FOODS,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,0
FOODS_3_004_CA_1_validation,FOODS_3_004,FOODS_3,FOODS,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,1,2,0,0,0,0,0,2,0,1
FOODS_3_005_CA_1_validation,FOODS_3_005,FOODS_3,FOODS,CA_1,CA,1,0,1,2,2,0,1,1,3,1,...,0,0,2,0,0,0,0,0,1,0


In [61]:
df_moderate = df_train_val.loc[median_series_idx].copy()
df_moderate = df_moderate.reset_index(level="item_id").set_index("item_id").T
df_moderate.index = df_moderate.index.map(day2date)
fig, axes = plt.subplots(nrows=df_moderate.shape[1], figsize=(12, 8))
df_moderate.plot(ax=axes, subplots=True)
for ax in axes.ravel():
    ax.legend(loc="upper left")
fig.suptitle("Moderate Time Series")
fig.autofmt_xdate()
fig.savefig(eda_path.joinpath("moderate_time_series.png"))
plt.close()

In [62]:
mask = np.triu(np.ones_like(df_moderate.corr(), dtype=bool))

# generate heatmap
fig, ax = plt.subplots(figsize=(6,3), layout="tight")
sns.heatmap(df_moderate.corr().round(3), annot=True, mask=mask, vmin=-1, vmax=1, ax=ax)
fig.autofmt_xdate(rotation=45)
ax.set_title('Correlation Matrix')
# plt.tight_layout()
fig.savefig(eda_path.joinpath("moderate_time_series_corr.png"))
plt.close()

In [63]:
df_moderate.corr()

item_id,FOODS_3_595,FOODS_3_669,FOODS_3_200,FOODS_3_566,FOODS_3_665
item_id,,,,,
FOODS_3_595,1.000000,0.115595,0.254753,0.211683,-0.006433
FOODS_3_669,0.115595,1.000000,0.091392,0.051119,0.044545
FOODS_3_200,0.254753,0.091392,1.000000,0.204855,0.022088
FOODS_3_566,0.211683,0.051119,0.204855,1.000000,-0.011879
FOODS_3_665,-0.006433,0.044545,0.022088,-0.011879,1.000000


In [64]:
delta = 1.0
dense_series_idx = (
    df_train_val
    .mean(axis=1)[lambda mean_demand: mean_demand.between(mean_demand_stats.loc["q_95"] - delta, mean_demand_stats.loc["q_95"] + delta)]
    .sample(5, replace=False)
    .index
)
dense_series_idx

MultiIndex([('FOODS_3_816_CA_3_validation', 'FOODS_3_816', 'FOODS_3', ...),
            ('FOODS_3_219_WI_1_validation', 'FOODS_3_219', 'FOODS_3', ...),
            ('FOODS_3_804_TX_1_validation', 'FOODS_3_804', 'FOODS_3', ...),
            ('FOODS_3_499_TX_2_validation', 'FOODS_3_499', 'FOODS_3', ...),
            ('FOODS_3_318_TX_3_validation', 'FOODS_3_318', 'FOODS_3', ...)],
           names=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])

In [65]:
df_dense = df_train_val.loc[dense_series_idx].copy()
df_dense = df_dense.reset_index(level="item_id").set_index("item_id").T
df_dense.index = df_dense.index.map(day2date)
fig, axes = plt.subplots(nrows=df_dense.shape[1], figsize=(12, 8))
df_dense.plot(ax=axes, subplots=True)
for ax in axes.ravel():
    ax.legend(loc="upper left")
fig.autofmt_xdate()
fig.suptitle("Dense Time Series")
fig.savefig(eda_path.joinpath("dense_time_series.png"))
plt.close()

In [66]:
mask = np.triu(np.ones_like(df_dense.corr(), dtype=bool))

# generate heatmap
fig, ax = plt.subplots(figsize=(6,3), layout="tight")
sns.heatmap(df_dense.corr().round(3), annot=True, mask=mask, vmin=-1, vmax=1, ax=ax)
fig.autofmt_xdate(rotation=45)
ax.set_title('Correlation Matrix')
# plt.tight_layout()
fig.savefig(eda_path.joinpath("dense_time_series_corr.png"))
plt.close()

In [55]:
df_dense.corr()

item_id,FOODS_3_816,FOODS_3_219,FOODS_3_804,FOODS_3_499,FOODS_3_318
item_id,,,,,
FOODS_3_816,1.000000,0.093463,0.036670,-0.039840,0.072558
FOODS_3_219,0.093463,1.000000,0.115766,0.009179,0.460425
FOODS_3_804,0.036670,0.115766,1.000000,-0.170153,0.088800
FOODS_3_499,-0.039840,0.009179,-0.170153,1.000000,-0.068616
FOODS_3_318,0.072558,0.460425,0.088800,-0.068616,1.000000


In [67]:
store_df = (
    df_train_val
    .xs("FOODS_3_200", level="item_id")
    .copy()
    .reset_index("store_id")
    .set_index("store_id")
    .T
)
store_df.index = store_df.index.map(day2date)
store_df.head()

store_id,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
2011-01-29,5,0,4,0,0,5,1,0,0,0
2011-01-30,1,0,4,0,1,10,3,0,0,0
2011-01-31,2,0,4,0,0,7,1,0,0,0
2011-02-01,0,0,5,0,1,4,1,0,0,0
2011-02-02,2,0,2,0,0,2,0,0,0,0


In [58]:
store_df.shape

(1913, 10)

In [70]:
def plot_demand_price_stores(
    df: pd.DataFrame, df_price: pd.DataFrame, df_calendar: pd.DataFrame,
    day2date: dict[str, pd.Timestamp], item_id: str, filepath: Optional[Path]=None
):
    df_stores = (
        df
        .xs(item_id, level="item_id")
        .copy()
        .reset_index("store_id")
        .set_index("store_id")
        .T
    )
    df_stores.index = df_stores.index.map(day2date)
    df_price_item = df_price.query(f"item_id == '{item_id}'").merge(df_calendar[["wm_yr_wk", "date"]], on="wm_yr_wk")
    nrows = 4
    ncols = 3
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 12), layout="tight")
    i = 0
    j = 0
    stores_name = list(df_stores.columns)
    for idx in range(nrows * ncols):
        j = idx // nrows
        i = idx - j * nrows
        if j > 0 and i == nrows - 1:  # only CA has 4 stores
            axes[i, j].remove() 
            continue
        store_id = stores_name.pop(0)
        df_store = pd.merge(
            df_stores[[store_id]],
            df_price_item.query(f"store_id == '{store_id}'")[["sell_price", "date"]],
            left_index=True,
            right_on="date",
            how="left"
        )
        df_store = df_store.fillna(value=0).set_index("date")
        ax = axes[i, j].twinx()
        df_store[store_id].plot(label="sales", ax=axes[i,j])
        df_store['sell_price'].plot(label="price", color="orange", ax=ax)
        handles = [
            Line2D([], [], color="blue", label="sales"),
            Line2D([], [], color="orange", label="price")
        ]
        axes[i, j].legend(handles=handles, frameon=True)
        axes[i, j].set_title(store_id)
        axes[i, j].set_ylabel("sales")
        ax.set_ylabel("price")
        axes[i, j].tick_params(axis="x", rotation=30)
    fig.suptitle(item_id)
    if filepath:
        fig.savefig(filepath.joinpath(f"{item_id}.png"))
        plt.close()
    else:
        plt.show()
    

In [73]:
plot_demand_price_stores(
    df_train_val, df_prices, df_calendar, day2date, item_id="FOODS_3_200", filepath=eda_path
)

In [74]:
plot_demand_price_stores(
    df_train_val, df_prices, df_calendar, day2date, item_id="FOODS_3_804", filepath=eda_path
)

In [67]:
df_train_val

,,,,,,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,item_id,dept_id,cat_id,store_id,state_id,,,,,,,,,,,,,,,,,,,,,
FOODS_3_001_CA_1_validation,FOODS_3_001,FOODS_3,FOODS,CA_1,CA,1,1,1,1,1,0,1,2,1,1,...,0,0,1,2,0,0,1,0,0,1
FOODS_3_002_CA_1_validation,FOODS_3_002,FOODS_3,FOODS,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FOODS_3_003_CA_1_validation,FOODS_3_003,FOODS_3,FOODS,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,1,0
FOODS_3_004_CA_1_validation,FOODS_3_004,FOODS_3,FOODS,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,1,2,0,0,0,0,0,2,0,1
FOODS_3_005_CA_1_validation,FOODS_3_005,FOODS_3,FOODS,CA_1,CA,1,0,1,2,2,0,1,1,3,1,...,0,0,2,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
